Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Potemkin Viktor"
COLLABORATORS = ""

---

# Large-scale eigenvalue problem with structured matrices


Consider a diagonal matrix $D$ of the size $n\times n$ and a column vector $u$ of size $n$. Consider a matrix 

$$
A = D +  \frac{u\, u^T}{u^T u}
$$

Drawing the elements of $D$ and $u$ from a standard normal distribution, find the smallest (by the absolute value) eigenvalue of $A$. You will need to consider values of up to $n=10^5$. Note that matrices of this size do not fit into memory, so you will need to write a matrix-free implementation which uses the special structure of $A$. You may want to consider using the Sherman-Morrison formula and the so-called [Bunch-Nielsen-Sorensen formula](https://en.wikipedia.org/wiki/Bunch%E2%80%93Nielsen%E2%80%93Sorensen_formula).

In [ ]:
import numpy as np
from numpy.testing import assert_allclose
from numpy.linalg import eigvals, eig

In [ ]:
from math import sqrt

def generate_d_u(n, rndm):
    """Generate two vectors of size $n$ drawn from a standard normal distribution."""
    d = rndm.standard_normal(size=n)
    u = rndm.standard_normal(size=n)
    u /= sqrt(u @ u)
    return d, u

We start from a simple case of small $n$, and write out the matrices explicitly.

In [ ]:
rndm = np.random.default_rng(12345)
d, u = generate_d_u(5, rndm)

A = np.diag(d) + u[:, None] @ u[None, :]
A

array([[-1.34292291,  0.14935839, -0.07085692, -0.03942634,  0.21324611],
       [ 0.14935839,  1.53946816, -0.13081332, -0.07278739,  0.39368675],
       [-0.07085692, -0.13081332, -0.80860276,  0.03453097, -0.18676843],
       [-0.03942634, -0.07278739,  0.03453097, -0.23995945, -0.10392203],
       [ 0.21324611,  0.39368675, -0.18676843, -0.10392203,  0.4867421 ]])

## Find the min eigenvalue of $A$

Min eigenvalue of $A$ is the max eigenvalue of $A^{-1}$. Use the power iteration with $A^{-1}$ as the iteration matrix.

Demo version: use full matrices for clarity. Also we only do a fixed number of iterations instead of a proper convergence check.

In [ ]:
def power_iter(A, x0, niter=20):
    """Compute min eigenvalue and corresponding eigenvector of matrix A.
    
    Use the power iteration.
    
    Parameters
    ----------
    A : ndarray of floats, shape (n, n)
        original matrix
    x0 : ndarray of floats, shape (n,)
        The starting vector for the iteration
    niter : int, optional
        the number of iterations (default is 20)
    
    Returns
    -------
    la : float
        min eigenvalue
    x : ndarray of floats, shape (n,)
        the iteration vector on the last iteration
    """
    xk = x0
    ykk = x0
    lam = 0
    a_inv = np.linalg.inv(A)
    for i in range(niter):
        ykk = a_inv @ xk
        lam = xk @ ykk
        xk = ykk / np.linalg.norm(ykk)
    x = xk
    la = 1/lam
    return la, x

In [ ]:
d, u = generate_d_u(5, rndm=np.random.default_rng(1234))
A = np.diag(d) + u[:, None] @ u[None, :]

la, x = power_iter(A, x0=np.ones(5))

# Check that the iteration vector approximates the eigenvector
np.linalg.norm(la * x - A @ x)

6.965336906136754e-14

In [ ]:
# this is a graded test

d1, u1 = generate_d_u(6, rndm)
A1 = np.diag(d1) + u1[:, None] @ u1[None, :]

la1, x1 = power_iter(A1, x0=np.ones(6))

err1 = np.linalg.norm(la1 * x1 - A1 @ x1)
assert_allclose(err1, 0., atol=1e-4)


## Check the BNS formula

Implement the [Bunch-Nielsen-Sorensen formula](https://en.wikipedia.org/wiki/Bunch%E2%80%93Nielsen%E2%80%93Sorensen_formula) for the matrix of the above structure. 

In [ ]:
def BNS(d, u, la):
    """Compute the eigenvector of $D + u u^T$ for a given eigenvalue.
    
    Given the matrix $A = D + u u^T$ where $D$ is a diagonal matrix and
    $u$ is a 1D vector, and also a precomputed eigenvalue of $A$,
    compute its corresponding eigenvector using the B-N-S formula.
    
    Parameters
    ----------
    d : ndarray, shape (n,)
        Diagonal elements of the matrix $D$
    u : ndarray, shape (n,)
        1D array representing the vector $u$
    la : float
        An eigenvalue of $A = D + u u^T$
        
    Returns
    -------
    q : ndarray, shape (n,)
        The normalized eigenvector of $A$, which corresponds
        to the eigenvalue `la`.
    """
    q = np.zeros(d.shape[0])
    for i in range(d.shape[0]):
        q[i] = u[i]/(d[i] - la)
    q = q / np.linalg.norm(q)
    return q

In [ ]:
# Note that the B-N-S vector $q$ looks to be a better approximation for the eigenvector.

d, u = generate_d_u(6, rndm=np.random.default_rng(1234))
A = np.diag(d) + u[:, None] @ u[None, :]
la, x = power_iter(A, x0=np.ones(6), niter=30)

q = BNS(d, u, la)

print(np.linalg.norm(la * x - A @ x))
print(np.linalg.norm(la * q - A @ q))

1.970586359267042e-06
7.162325834083003e-10


In [ ]:
q1 = BNS(d1, u1, la1)
assert np.linalg.norm(la1 * q1 - A1 @ q1) < 1e-9


## Implement the the matrix-free version of the above prototype


First, implement the function which computes the result of the matrix-vector product of the matrix 

$$\left(D + \dfrac{u\, u^T}{u^T u} \right)^{-1}$$

and a given vector $x$. Your implementation must never construct the matrix explicitly. Use the [Sherman-Morrison formula](https://en.wikipedia.org/wiki/Sherman%E2%80%93Morrison_formula).

In [ ]:
def SMproduct(d, u, x):
    """Compute the matrix-vector product $(D + u u^T)^{-1} x$.
    
    Here $D$ is the diagonal matrix with `d` on the main diagonal,
    and `u` is a 1D array representing the (column) vector $u$.
    
    `x` is also a 1D array representing the $x$ vector.
    
    Use the Sherman-Morrison formula.
    
    Parameters
    ----------
    d : ndarray of floats, shape (n,)
        $D$ is the diagonal matrix with `d` on the main diagonal
    u : ndarray of floats, shape (n,)
        a 1D array representing the vector $u$
    x : ndarray of floats, shape (n,)
        a 1D array representing the vector $x$
    
    Returns
    -------
    y : array-like of floats, shape (n,)
        The result of the matrix-vector product of $D+(u u^T)^{-1} x$
    """
    x_div = x / d
    u_div = u / d
    factor = u_div @ x
    den = 1 + u @ u_div
    res = x_div - u_div * factor / den
    return res

In [ ]:
# Test your matrix-vector product routine on small size matrices

n = 5
rndm = np.random.default_rng(1234)

d, u = generate_d_u(n, rndm)
A = np.diag(d) + u[:, None] @ u[None, :]

for x in [np.arange(n), np.ones(n), rndm.uniform(size=n)]:
    assert_allclose(SMproduct(d, u, x),
                    np.linalg.solve(A, x), atol=1e-10)

## Compute the leading eigenvalue of $(D + u u^T)^{-1}$.

Write a function which computes the leading eigenvalue and its corresponding eigenvector.

In [ ]:
def leading_eigv(d, u, x0, eps=1e-6, maxiter=1000):
    """Compute the leading eigenvalue of (D + u@u.T)^{-1}.
    
    Use the power iteration method to compute the eigenvalue.
    
    Use the B-N-S formula to compute the corresponding eigenvector.
    
    Parameters
    ----------
    d : array, shape (n,)
        The diagonal elements
    u : array, shape (n,)
        The outer product vector
    x0 : array, shape (n,)
        The initial approximation
    eps : float
        The target accuracy.
        Iterations stop when |true_lambda - current estimate| < eps.
    maxiter : int
        The maximum allowed number of iterations.
    
    Returns
    -------
    lmbda : float
        The estimate for the leading eigenvalue
    vec : array, shape (n,)
        The estimate for the corresponding eigenvector
        
    Raises
    ------
    ValueError
        if the maximum number of iterations `maxiter` is reached
        without reaching the target tolerance.
    
    """
    xk = x0
    ykk = x0
    lam = 0
    lam_new = 0
    for i in range(maxiter):
        ykk = SMproduct(d, u, xk)
        lam = lam_new
        lam_new = xk @ ykk
        xk = ykk / np.linalg.norm(ykk)
        if np.linalg.norm(SMproduct(d, u, xk) - lam_new*xk) < eps:
            break
    else:
        raise ValueError("target tolerance is not reached in %s iterations" % maxiter)
    la = lam_new
    x = BNS(d, u, 1/la)
    return la, x

In [ ]:
#my own test to experiment with code
n = 5
rndm = np.random.default_rng(1234)
d, u = generate_d_u(n, rndm)

la1, x = leading_eigv(d, u, np.ones(n))
A = np.diag(d) + u[:, None] @ u[None, :]
print(np.linalg.eig(A)[0], la1)
print(np.linalg.norm(SMproduct(d, u, x) - la1*x))

[-1.15502282  0.09895118  0.47297607  0.96089908  0.83971397] 10.105993793184826
9.987023267832838e-13


In [ ]:
# test on a small example (useful for debugging)
n = 5
rndm = np.random.default_rng(1234)
d, u = generate_d_u(n, rndm)

la1, x = leading_eigv(d, u, np.ones(n))
assert np.linalg.norm(SMproduct(d, u, x) - la1*x) < 1e-9

e = eigvals(np.diag(d) + u[:, None] @ u[None, :])
e = e[np.argsort(abs(e))]
assert_allclose(e[0], 1./la1, atol=1e-14, rtol=0)

In [ ]:
# test interface
import pytest
with pytest.raises(ValueError):
    la1, x = leading_eigv(d, u, np.ones(n), eps=1e-20, maxiter=1)

In [ ]:
# test on a slightly larger example
n = 100
rndm = np.random.default_rng(1234)
d, u = generate_d_u(n, rndm)

la1, x = leading_eigv(d, u, np.ones(n))
assert np.linalg.norm(SMproduct(d, u, x) - la1*x) < 1e-9

# and a yet larger one
n = int(1e5)
rndm = np.random.default_rng(1234)
d, u = generate_d_u(n, rndm)

la1, x = leading_eigv(d, u, np.ones(n))
assert np.linalg.norm(SMproduct(d, u, x) - la1*x) < 1e-9